In [52]:
import os
import numpy as np
from collections import Counter
import re

cwd = (os.getcwd())
log = ("/auth.log")
path = (cwd+log)

with open(path) as f:
    f = f.readlines()
logs_list = []
for line in f:
    logs_list.append(line)
    
user_failed = []   
user_accepted = [] 

ip_adresses_failed = []
ip_adresses_accepted = []

result_failed = []
result_accepted = []

the_lines_I_need = []
splits = []

for line in logs_list:
    if "from" in line:  
        the_lines_I_need.append(line)
          

for l_1 in the_lines_I_need: 
    if "Failed password " in l_1:
        user_failed = re.findall(r"([for]+\s\w*\s[from]+)", l_1)
        for f in user_failed:
            a,b,c = f.split( )
            splits.append(b)
            
        ip_adresses_failed = re.findall(r"[0-9]+(?:\.[0-9]+){3}", l_1)
        result_failed.append(b)
        result_failed.extend(ip_adresses_failed)


for l_2 in the_lines_I_need:
    if "Accepted password" in l_2:
        user_accepted = re.findall(r"([for]+\s\w*\s[from]+)", l_2)
        for a in user_accepted:
            a,b,c = a.split( )
            splits.append(b)
            
        ip_adresses_accepted = re.findall(r"[0-9]+(?:\.[0-9]+){3}", l_2)
        result_accepted.append(b)
        result_accepted.extend(ip_adresses_accepted)


failed_list = [result_failed[i:i+2] for i in range(0, len(result_failed), 2)]
narray_f = np.asarray(failed_list)


accepted_list = [result_accepted[i:i+2] for i in range(0, len(result_accepted), 2)]
narray_a = np.asarray(accepted_list)

#Failed by Ip adress, f.e: 10.0.0.202 - 19
Count_uni_failed_per_IP_List = [t2 for t1, t2 in failed_list ]
failed_per_IP_list_f = Counter(Count_uni_failed_per_IP_List)
for key in sorted(failed_per_IP_list.iterkeys()):
    failed_per_IP_list_f.append([key, c[key]])


unq1, count1 = np.unique(narray_f[:,1], axis=0, return_counts=True)
failed_per_IP = np.column_stack((unq1, count1))

#Accepted by Ip adress, f.e: 1 accepted for 1 users
unq2, count2 = np.unique(narray_a[:,1], axis=0, return_counts=True)
accepted_per_IP = np.column_stack((unq2, count2))

#Failed by users, f.e: failed for 1 users
unq3, count3 = np.unique(narray_f, axis=0, return_counts=True)
stacked_f = np.column_stack((unq3, count3))

unq4, count4 = np.unique(stacked_f[:,1], axis=0, return_counts=True)
failed_users_per_IP = np.column_stack((unq4, count4))

matrix_f = np.column_stack((failed_users_per_IP, failed_per_IP[:,1]))

#Accepted by users, f.e: 1 accepted for 1 users
unq5, count5 = np.unique(tat_accepted, axis=0, return_counts=True)
stacked_a = np.column_stack((unq5, count5))

unq6, count6 = np.unique(stacked_a[:,1], axis=0, return_counts=True)
accepted_users_per_IP = np.column_stack((unq6, count6))

matrix_a = np.column_stack((accepted_users_per_IP, accepted_per_IP[:,1]))

ips = []
for ip in matrix_a[:,0]:
    if ip in matrix_f[:,0]:
        ips.append(ip)
        
final_a = [] 
for i in matrix_a:
    for s in ips:
        if i[0] == s:
            final_a.append(i)

final_f = [] 
for i in matrix_f:
    for s in ips:
        if i[0] == s:
            final_f.append(i)

matrix_final = np.column_stack((final_a, final_f)) 

    
for i in matrix_final:
    if int(i[5]) > int(i[2]) >= 1:
        print("{0} - {1} failed for {2} users - {3} accepted for {4} users".format(i[0],i[5],i[4],i[2],i[1]))
        

AttributeError: 'Counter' object has no attribute 'iterkeys'